# Segmenting and Clustering Neighborhoods in Toronto (Scraping Webpages)
<hr>
The objective of this notebook can be summarized as:<br>
- Use the Notebook to build the code to scrape the following Wikipedia page, <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"> https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada </a>, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
   
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood, <br>
- Only process the cells that have an assigned borough, <br>
- Ignore cells with a borough that is Not assigned, <br>
- More than one neighborhood can exist in one postal code area. Combine these rows into one row with neighborhoods separated with a comma, <br>
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough, <br>

In [22]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [23]:
!conda install -c anaconda beautifulsoup4
!conda install -c anaconda html5lib 

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: - ^C
failed

CondaError: KeyboardInterrupt

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
^C

CondaError: KeyboardInterrupt



#### Specific libraries required for scraping websites
- __Beautiful Soup__ is a library for pulling data out of HTML and XML files. It provides ways of navigating, searching, and modifying parse trees. Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.
- __html5lib__ is a pure-python library for parsing HTML. It is designed to conform to the WHATWG HTML specification, as is implemented by all major web browsers.
- __Requests__ allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.

# Ping the wiki page and get the data:

In [24]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# website_url

 # Reading the source code for a given web page and creating a BeautifulSoup (soup)object
 - __Prettify() function__ in BeautifulSoup will enable us to view how the tags are nested in the document.
 - If you carefully inspect the HTML script all the table contents i.e. names of the countries which we intend to extract is under class Wikitable Sortable.<br>

In [25]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
# print(soup.prettify())

- So our first task is to find class ‘wikitable sortable’ in the HTML script.
- Also note: Under table class ‘wikitable sortable’ all the entries for the table like postalcode/Borough name/Neighborhood are stored within < td > .... < /td >

In [26]:
toronto_tab = soup.find('table',{'class':'wikitable sortable'})
tds = toronto_tab.findAll('td')

stack = []
for tr in toronto_tab.findAll('tr'):
    for td in tr.findAll('td'):
        stack.append(td.text.strip())
# print(stack)

# Convert the scraped data into Pandas DataFrame to work in python.

In [27]:
Postcode_list=[]
Borough_list=[]
Neighborhood_list=[]
num_rows = len(stack)
num_rows = int(num_rows)

for i in range(0,num_rows,3):
    Postcode_list.append(stack[i])
    Borough_list.append(stack[i+1])
    Neighborhood_list.append(stack[i+2])

In [28]:
toronto_df=pd.DataFrame()
toronto_df['Postcode']=Postcode_list
toronto_df['Borough']=Borough_list
toronto_df['Neighborhood']=Neighborhood_list
print('The shape of toronto_df is {}'.format(toronto_df.shape))
toronto_df.head()

The shape of toronto_df is (288, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Clean the dataset as per the guidelines mentioned in the introduction.

In [29]:
toronto_df_NAN = toronto_df.replace('Not assigned', np.nan, regex=True)
toronto_df_NAN.head(3)

,Postcode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods


- Removing all rows for which the Borough value is not assigned.

In [30]:
toronto_noNAN = toronto_df_NAN.dropna(subset = ['Borough'])
print('The size of toronto_noNAN is {}'.format(toronto_noNAN.shape))
toronto_df_noNAN = toronto_noNAN.reset_index(drop=True)
toronto_df_noNAN.head(7)

The size of toronto_noNAN is (211, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,NaN


- Putting Neighborhood value as Borough value if only neighborhood value was not assigned.

In [31]:
toronto_df_noNAN = toronto_df_noNAN.replace(np.nan, 'Not assigned', regex=True)

Borough_listN = toronto_df_noNAN['Borough'].tolist()
Neighborhood_listN = toronto_df_noNAN['Neighborhood'].tolist()

for j in range(0,len(Borough_listN),1):
    if(Neighborhood_listN[j]=='Not assigned'):
        Neighborhood_listN[j]=Borough_listN[j]
        
toronto_df_noNAN['Borough']=Borough_listN
toronto_df_noNAN['Neighborhood']=Neighborhood_listN

toronto_df_noNAN = toronto_df_noNAN.replace('Not assigned', np.nan, regex=True)
toronto_df_noNAN.isnull().values.any() # checks presence of np.nan in the entire dataset

False

In [32]:
toronto_df_noNAN.head(7)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


- Combining rows when more than one neighborhood exists in one postal code area.

In [33]:
duplicate = toronto_df_noNAN.duplicated(subset=['Postcode'], keep='first') # assigns True to all the first, second .... duplicates of Postcode values
duplicate.head()

0    False
1    False
2    False
3     True
4    False
dtype: bool

In [34]:
toronto_cleanDF = toronto_df_noNAN

todrop_list = []
for k in range(0,len(Borough_listN),1):
    if(duplicate[k]):
        temp_var = Neighborhood_listN[k-1]
        temp_var = temp_var + ', ' + Neighborhood_listN[k]
        Neighborhood_listN[k] = temp_var
        todrop_list.append(k-1)
        
toronto_cleanDF['Neighborhood'] = Neighborhood_listN

In [35]:
print(todrop_list)

[2, 4, 8, 11, 13, 16, 17, 18, 19, 21, 22, 24, 29, 30, 31, 33, 34, 45, 46, 49, 50, 52, 55, 56, 58, 61, 62, 64, 66, 67, 70, 72, 74, 76, 77, 79, 80, 82, 85, 87, 89, 90, 93, 94, 96, 100, 102, 103, 104, 106, 108, 114, 117, 118, 122, 124, 127, 131, 132, 134, 137, 138, 139, 143, 145, 147, 148, 150, 152, 153, 155, 156, 157, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 176, 177, 178, 179, 180, 185, 189, 191, 193, 194, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209]


In [36]:
toronto_DF = toronto_cleanDF.drop(todrop_list)
toronto_DF = toronto_DF.reset_index(drop=True)

In [37]:
toronto_DF.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


- ## The final shape of the data set is:

In [38]:
toronto_DF.shape

(103, 3)

# To utilize Foursquare location data, we need latitude & longitude coordinates of each neighborhood.

In [42]:
!pip install geocoder
# !conda install -c conda-forge geocoder # Geocoder Python package to get the latitude and the longitude coordinates of each neighborhood

    100% |████████████████████████████████| 102kB 18.5MB/s 


In [43]:
import geocoder # import geocoder

Latitude = []
Longitude = []

for bor in (toronto_DF['Borough'].tolist()):
    lat_lng_coords = None # initialize your variable to None
    g = geocoder.arcgis(bor)
    lat_lng_coords = g.latlng
    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])

In [44]:
data = pd.read_csv('https://cocl.us/Geospatial_data') # used this as some of the locations were outside Toronto

In [45]:
# toronto_new = toronto_DF
# toronto_new['Latitude'] = Latitude
# toronto_new['Longitude'] = Longitude
# toronto_new.head()

In [46]:
toronto_new = toronto_DF
toronto_new.sort_values(by=['Postcode'], inplace=True)
toronto_new = toronto_new.reset_index()

toronto_new['Latitude'] = data['Latitude']
toronto_new['Longitude'] = data['Longitude']

toronto_new.drop("index", axis=1, inplace=True)
toronto_new.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Analyzing each neighborhood using FOURSQUARE location data

In [47]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [49]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_new['Latitude'], toronto_new['Longitude'], toronto_new['Borough'], toronto_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [50]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(toronto_new['Borough'].unique()), toronto_new.shape[0]))
toronto_new['Borough'].unique()

The dataframe has 11 boroughs and 103 neighborhoods.


array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

### Map of Neighborhoods in Downtown Toronto, but later we will consider all neighborhoods for exploration

In [51]:
downtown_data = toronto_new[toronto_new['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [52]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [53]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Define Foursquare Credentials and Version

In [54]:
CLIENT_ID = 'NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW' # your Foursquare ID
CLIENT_SECRET = 'FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NY1X4ZTFDKEJY13TWNTNBPSEBCC4AVAG3WLXZVP4YMMT4LRW
CLIENT_SECRET:FSEH2VNFCDNZ4ZZKITQU1SBFO5H0KQ0XKOG54KK4K2CZMM2T


In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
#     return(nearby_venues)

# type your answer here

venues_df = getNearbyVenues(names=toronto_new['Neighborhood'],
                                   latitudes=toronto_new['Latitude'],
                                   longitudes=toronto_new['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [64]:
print(venues_df.shape)
venues_df.head()

""


In [109]:
# trial = venues_df.groupby('Neighborhood').count()
# trial.shape
# venues_df.isnull().values.any() # checks presence of np.nan in the entire dataset

In [59]:
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

TypeError: 'NoneType' object is not subscriptable

### Analyze each neighborhood

In [111]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped
# toronto_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, Ki

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]): # iterates from 0 ... 39 the number of neighborhoods in Mangattan_grouped
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    # manhattan_grouped.iloc[x, y] where x represent the row while y takes values from 0....333

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Sushi Restaurant,Hotel,Bakery,Bar,Burger Joint
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Skating Rink,Chinese Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Deli / Bodega
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Fast Food Restaurant,Pharmacy,Pizza Place,Fried Chicken Joint,Beer Store,Sandwich Place,Discount Store,Department Store,Dessert Shop
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Dance Studio,Pharmacy,Sandwich Place,Skating Rink,Pub,Dim Sum Restaurant,Deli / Bodega


## 6. Segmenting and Clustering neighborhoods in Toronto

In [114]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_new # manhattan_data is separated from original data using Borough='Manhattan'

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Golf Course,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Spa,Pizza Place,Rental Car Location,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Diner,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Korean Restaurant,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Fried Chicken Joint,Caribbean Restaurant,Bakery,Bank,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Drugstore,Donut Shop,Doner Restaurant


In [123]:
toronto_merged[['Cluster Labels']] = toronto_merged[['Cluster Labels']].astype(int)

In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 7. Examine Clusters
Examine each cluster to determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

### Cluster 1

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,0,Baseball Field,Construction & Landscaping,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Women's Store
97,North York,0,Baseball Field,Construction & Landscaping,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Women's Store


### Cluster 2

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1,Pizza Place,Playground,Women's Store,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
81,York,1,Convenience Store,Pizza Place,Bus Line,Caribbean Restaurant,Women's Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
95,Etobicoke,1,Liquor Store,Pizza Place,Beer Store,Convenience Store,Café,Pharmacy,College Stadium,Deli / Bodega,Eastern European Restaurant,College Rec Center
96,North York,1,Empanada Restaurant,Pizza Place,Dive Bar,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


### Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]